# Imports/Setup

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from IPython.display import clear_output
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

%matplotlib inline
load_dotenv()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv('CLIENT_ID'), CLIENT_SECRET=os.getenv('CLIENT_SECRET')))

# Download Data

## How many genres are there?

In [2]:
genres = sp.recommendation_genre_seeds()['genres']
print(len(genres), 'genres')

126 genres


## Get set of tracks, top 100 per genre

In [3]:
tracks = []
for i, g in enumerate(genres):
    clear_output(wait=True)
    print(i, g)
    recs = sp.recommendations(seed_genres=[g], limit=100)['tracks']
    recs_id_name_artist = [(track['id'], track['name'], track['artists'][0]['name'], g) for track in recs]
    for track in recs_id_name_artist:
        tracks.append(track)
clear_output(wait=True)
print('Tracks:', len(tracks))

Tracks: 12526


## Get audio features per track

In [4]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

track_chunks = list(chunks(tracks, 100))
audio_features = []
for c in track_chunks:
    c = [ch for ch in c if ch[0] is not None]
    c_ids = [track[0] for track in c]
    c_names = [track[1] for track in c]
    c_artists = [track[2] for track in c]
    c_genres = [track[3] for track in c]
    c_audio_features = sp.audio_features(tracks=c_ids)
    for i, _ in enumerate(c_audio_features):
        c_audio_features[i]['name'] = c_names[i]
        c_audio_features[i]['artist'] = c_artists[i]
        c_audio_features[i]['genre'] = c_genres[i]
    audio_features += c_audio_features
print('Audio Features:', len(audio_features))

Audio Features: 12526


## Save to CSV

In [5]:
track_df = pd.DataFrame(audio_features)
track_df.to_csv('../data/other.csv')
track_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,name,artist,genre
0,0.640,0.329,11,-10.844,0,0.0323,0.8960,0.000536,0.1130,0.2950,...,audio_features,7AYoXqCtME90flUOpBJM7i,spotify:track:7AYoXqCtME90flUOpBJM7i,https://api.spotify.com/v1/tracks/7AYoXqCtME90...,https://api.spotify.com/v1/audio-analysis/7AYo...,236307,4,Society,Eddie Vedder,acoustic
1,0.503,0.294,8,-11.439,0,0.1000,0.8910,0.000000,0.0983,0.0974,...,audio_features,3KnQSveB0TCz2Lu3TaJWMJ,spotify:track:3KnQSveB0TCz2Lu3TaJWMJ,https://api.spotify.com/v1/tracks/3KnQSveB0TCz...,https://api.spotify.com/v1/audio-analysis/3KnQ...,225427,4,Walking on a Dream,Kate Miller-Heidke,acoustic
2,0.716,0.296,7,-12.145,1,0.0360,0.4910,0.000011,0.1130,0.5420,...,audio_features,5d8L73s2DVUlIi4KENcxeO,spotify:track:5d8L73s2DVUlIi4KENcxeO,https://api.spotify.com/v1/tracks/5d8L73s2DVUl...,https://api.spotify.com/v1/audio-analysis/5d8L...,252227,4,More Than This,Charlie Hunter,acoustic
3,0.674,0.428,7,-9.504,1,0.1220,0.1210,0.000000,0.1320,0.3370,...,audio_features,2dLLR6qlu5UJ5gk0dKz0h3,spotify:track:2dLLR6qlu5UJ5gk0dKz0h3,https://api.spotify.com/v1/tracks/2dLLR6qlu5UJ...,https://api.spotify.com/v1/audio-analysis/2dLL...,190185,4,Royals,Lorde,acoustic
4,0.640,0.394,5,-7.258,1,0.0263,0.6950,0.000000,0.0741,0.4250,...,audio_features,0vWjNkwgkTnyzfeLdTKWJ7,spotify:track:0vWjNkwgkTnyzfeLdTKWJ7,https://api.spotify.com/v1/tracks/0vWjNkwgkTny...,https://api.spotify.com/v1/audio-analysis/0vWj...,196362,4,Roar,Alex Goot,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12521,0.396,0.544,6,-8.253,0,0.0340,0.2820,0.495000,0.1110,0.2960,...,audio_features,6Ru1MJhTTFFnbSUqTGmrtM,spotify:track:6Ru1MJhTTFFnbSUqTGmrtM,https://api.spotify.com/v1/tracks/6Ru1MJhTTFFn...,https://api.spotify.com/v1/audio-analysis/6Ru1...,305880,3,Adounia (Life),Bombino,world-music
12522,0.785,0.743,9,-3.364,1,0.0441,0.0585,0.079900,0.1390,0.6980,...,audio_features,6nrUDtBBWBKuSDofJpphDd,spotify:track:6nrUDtBBWBKuSDofJpphDd,https://api.spotify.com/v1/tracks/6nrUDtBBWBKu...,https://api.spotify.com/v1/audio-analysis/6nrU...,263813,4,Epoca,Gotan Project,world-music
12523,0.749,0.592,10,-11.633,1,0.0338,0.1080,0.514000,0.0659,0.9240,...,audio_features,7nwjP8zLSNPGU2d99d59Uc,spotify:track:7nwjP8zLSNPGU2d99d59Uc,https://api.spotify.com/v1/tracks/7nwjP8zLSNPG...,https://api.spotify.com/v1/audio-analysis/7nwj...,290840,4,Ahe Sira Bila,Issa Bagayogo,world-music
12524,0.637,0.837,6,-5.083,1,0.0318,0.0572,0.000005,0.0421,0.6530,...,audio_features,5kNqKS5CCrT94IRxuXnDCx,spotify:track:5kNqKS5CCrT94IRxuXnDCx,https://api.spotify.com/v1/tracks/5kNqKS5CCrT9...,https://api.spotify.com/v1/audio-analysis/5kNq...,195173,4,Ride with Yourself,Rhino Bucket,world-music


# DB Upload

In [6]:
conn = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(os.getenv('DB_USER'),
                                                               os.getenv('DB_PASS'), 
                                                               os.getenv('DB_IP'), 
                                                               os.getenv('DB_PORT'), 
                                                               os.getenv('DB_NAME')))
track_df.to_sql('other', con=conn, if_exists='replace')